In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [7]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [9]:
if __name__ == '__main__':  # Ensures the script runs only if executed directly, not imported.
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36', 
                'Accept-Language': 'en-US, en;q=0.5'})  # Headers to mimic a real browser request to avoid bot detection.
    
    URL = "https://www.amazon.com/s?k=headphones&crid=2CNZSNP0C3Z6D&sprefix=headphone%2Caps%2C466&ref=nb_sb_noss_2"  # URL for the Amazon search page.

    webpage = requests.get(URL, headers=HEADERS)  # Sends a GET request to fetch the webpage content.
    
    soup = BeautifulSoup(webpage.content, "html.parser")  # Parses the webpage content using BeautifulSoup.

    links = soup.find_all("a", attrs={'class':'a-link-normal s-line-clamp-2 s-link-style a-text-normal'})  # Finds all product links.

    links_list = []  # Initializes an empty list to store product links.
    
    for link in links:
        links_list.append(link.get('href'))  # Extracts and appends each product link to the list.

    d = {"title":[], "price":[], "rating":[], "reviews":[], "availability":[]}  # Initializes a dictionary to store product details.

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)  # Sends request for each product page.
        
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")  # Parses the product page content.

        # Function calls to extract and append product details to the dictionary.
        d['title'].append(get_title(new_soup))  # Extracts product title.
        d['price'].append(get_price(new_soup))  # Extracts product price.
        d['rating'].append(get_rating(new_soup))  # Extracts product rating.
        d['reviews'].append(get_review_count(new_soup))  # Extracts number of reviews.
        d['availability'].append(get_availability(new_soup))  # Extracts product availability status.

    amazon_df = pd.DataFrame.from_dict(d)  # Converts the dictionary into a Pandas DataFrame.

    amazon_df['title'].replace('', np.nan, inplace=True)  # Replaces empty titles with NaN values.

    amazon_df = amazon_df.dropna(subset=['title'])  # Drops rows where the title is NaN.

    amazon_df.to_csv("amazon_data.csv", header=True, index=False)  # Saves the data to a CSV file.


C:\Users\prashant\AppData\Local\Temp\ipykernel_15188\3047839547.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)  # Replaces empty titles with NaN values.


In [5]:
amazon_df

,title,price,rating,reviews,availability
0,Beats Studio Pro - Wireless Bluetooth Noise Ca...,,4.5 out of 5 stars,"17,697 ratings",Not Available
1,Sony WH-CH520 Wireless Headphones Bluetooth On...,,4.7 out of 5 stars,"18,089 ratings",Not Available
2,Sony WH-CH720N Noise Canceling Wireless Headph...,,4.7 out of 5 stars,"8,959 ratings",Not Available
3,Soundcore Anker Life Q20 Hybrid Active Noise C...,,4.7 out of 5 stars,"84,762 ratings",Not Available
4,"Apple AirPods 4 Wireless Earbuds, Bluetooth He...",,4.6 out of 5 stars,"3,173 ratings",Not Available
5,"Bose QuietComfort Bluetooth Headphones, Wirele...",,4.5 out of 5 stars,"9,090 ratings",Not Available
6,JBL Tune 770NC - Adaptive Noise Cancelling wit...,,4.5 out of 5 stars,"1,700 ratings",Not Available
7,Beats Studio Buds - True Wireless Noise Cancel...,,4.4 out of 5 stars,"82,228 ratings",Not Available
8,OneOdio Wired Over Ear Headphones Hi-Res Studi...,,4.5 out of 5 stars,"70,611 ratings",Not Available
9,Sony WH-1000XM4 Wireless Premium Noise Canceli...,,4.7 out of 5 stars,"59,192 ratings",Not Available
